In [1]:
from pyspark.sql import SparkSession

DIRPATH = "C:/Users/sanjay s risbud/Downloads/sundog/"
spark = SparkSession.builder.master("local").appName("Exercise").getOrCreate()

In [7]:
from pyspark.sql.functions import split
lines = spark.read.text(DIRPATH + "/ml-100k/u.data")
df = lines.withColumn("parsed", split(lines["value"], "\s+"))
df = df.withColumn("movie_id", df["parsed"][1]).withColumn("user_id", df["parsed"][0]).select(["movie_id", "user_id"])

+--------+-------+
|movie_id|user_id|
+--------+-------+
|     242|    196|
|     302|    186|
|     377|     22|
|      51|    244|
|     346|    166|
|     474|    298|
|     265|    115|
|     465|    253|
|     451|    305|
|      86|      6|
|     257|     62|
|    1014|    286|
|     222|    200|
|      40|    210|
|      29|    224|
|     785|    303|
|     387|    122|
|     274|    194|
|    1042|    291|
|    1184|    234|
+--------+-------+
only showing top 20 rows



In [20]:
from pyspark.sql.functions import desc
popular_list = df.groupBy("movie_id").count().orderBy(desc("count")).head(10)
popular_df = spark.createDataFrame(popular_list)
popular_df.show()

+--------+-----+
|movie_id|count|
+--------+-----+
|      50|  583|
|     258|  509|
|     100|  508|
|     181|  507|
|     294|  485|
|     286|  481|
|     288|  478|
|       1|  452|
|     300|  431|
|     121|  429|
+--------+-----+



In [31]:
movies_df = spark.read.csv(DIRPATH + "ml-100k/u.item", sep="|")
movies_df = movies_df.select(movies_df["_c0"].alias("movie_id"), movies_df["_c1"].alias("name"))
movies_df.join(popular_df, on="movie_id").select(["name", "count"]).show(truncate=False)

+-----------------------------+-----+
|name                         |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+



In [32]:
spark.stop()